In [ ]:
import json
import logging
import os
from  nltk import ngrams
import multiprocessing as mp
import matplotlib.pyplot
from math import log

In [ ]:
def load_doc(filename):
    file = open(filename, 'r', encoding='utf-8-sig')
    text = file.read()
    file.close()
    return text

In [ ]:
# gender pair list taken from GN-Glove
DEFAULT_MALE_NOUNS = load_doc('male_word_file.txt').split('\n')
DEFAULT_FEMALE_NOUNS = load_doc('female_word_file.txt').split('\n')

In [ ]:
def gender_ratios_m_f(output_data_dir,file, model,maleCount, femaleCount):
    scoresP = []
    scoresPC = []
    with open(file,'r') as f:
        data = json.load(f)
    bias_record = {}
    for words in data:
        if (data[words]['m']+data[words]['f'] > MinCount):
            if words not in DEFAULT_MALE_NOUNS and words not in DEFAULT_FEMALE_NOUNS:
                score = abs(log((data[words]['m']+1)/(data[words]['f']+1)))
                scoresP.append(score)
                pm = data[words]['m'] / maleCount
                pf = data[words]['f'] / femaleCount
                score_c = abs(log((pm+0.0001)/(pf+0.0001)))
                scoresPC.append(score_c)
    output_file = os.path.join(output_data_dir, 'biased_words_m_f' + model)   
    with open(output_file,'w') as fp:
        json.dump(bias_record,fp, sort_keys=True) 
    print('words considered: ',len(scoresP))
    print("Bias_score_P: ", (sum(scoresP)/len(scoresP)))
    print("Bias_score_PC: ", (sum(scoresPC)/len(scoresPC)))
    return scoresP,scoresPC

In [ ]:
# Taken from jtcrammer repo
def get_cooccurrences(file, data, window):           
       
    with open(file, 'r',encoding='utf-8-sig') as fp:
        sentences = fp.read()
        
    male_nouns = DEFAULT_MALE_NOUNS
    female_nouns = DEFAULT_FEMALE_NOUNS
    n_grams = ngrams(sentences.split(), window)
    
    for grams in n_grams:
        pos = 1
        m = 0 
        f = 0 
        for w in grams:
                pos+=1
                if w not in data:
                    data[w]= {"m":0, "f":0}
                
                if pos==int((window+1)/2):
                    if w in male_nouns:
                        m = 1
                    if w in female_nouns:
                        f = 1
                    if m > 0:
                        for t in grams:
                            if t not in data:
                                data[t]= {"m":0, "f":0}
                            data[t]['m']+=1
                    if f > 0:
                        for t in grams:
                            if t not in data:
                                data[t]= {"m":0, "f":0}
                            data[t]['f']+=1
    return data

In [ ]:
def getGenderCounts(file):
    with open(file, 'r',encoding='utf-8-sig') as fp:
        sentences = fp.read()
    tokens = sentences.split()
    maleCount = len([w for w in tokens if w in DEFAULT_MALE_NOUNS])
    femaleCount = len([w for w in tokens if w in DEFAULT_FEMALE_NOUNS])
    return maleCount,femaleCount

In [ ]:
output_data_dir = ''
txtDir = ''
models = os.listdir(txtDir)
models

In [ ]:
window=10
MinCount = 20

In [ ]:
biasPList = []
biasPCList = []
scorePList = []
scorePCList = []
maleRatio = []
for m in models:
    data ={}
    print("Processing: ",m)
    print("Files:",len(os.listdir(txtDir + m)))
    maleCount = 0
    femaleCount = 0
    for file in os.listdir(txtDir + m):
        txt_path = txtDir + m + '/' + file
        data = get_cooccurrences(txt_path, data, window)
        mCount,fCount = getGenderCounts(txt_path)
        maleCount += mCount
        femaleCount += fCount
    output_file = output_data_dir + m + '.json'
    with open(output_file,'w') as fp:
        json.dump(data,fp)
    print('male word ratio :', maleCount/(maleCount+femaleCount))
    maleRatio.append(maleCount/(maleCount+femaleCount))
    scores_p,scores_pc = gender_ratios_m_f(output_data_dir,output_file, m, maleCount, femaleCount)
    scorePList.append(scores_p)
    scorePCList.append(scores_pc)
    biasPList.append(sum(scores_p)/len(scores_p))
    biasPCList.append(sum(scores_pc)/len(scores_pc))

In [ ]:
## Just arranging model names by lambda
mm = ['lambda_0','cda', 'lambda_0.01','lambda_0.1', 'lambda_0.5', 'lambda_0.8',\
      'lambda_1', 'lambda_2', 'lambda_5','lambda_10']
scorePList = [dict(zip(models,scorePList))[m] for m in mm]
scorePCList = [dict(zip(models,scorePCList))[m] for m in mm]
biasPList = [dict(zip(models,biasPList))[m] for m in mm]
biasPCList = [dict(zip(models,biasPCList))[m] for m in mm]
maleRatio = [dict(zip(models,maleRatio))[m] for m in mm]
modelsNames = mm

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(4,2))
plt.bar(modelsNames, biasPList)
plt.ylabel('bias')
plt.title('Avg abs( log( p(w,m) / p(w,f) ))')
_=plt.xticks(modelsNames, rotation='vertical')

plt.figure(figsize=(4,2))
plt.bar(modelsNames, biasPCList, color=(0.6, 0.8, 0.5, 1))
plt.title('Avg abs( log( p(w|m) / p(w|f) ))')
plt.ylabel('bias')
_=plt.xticks(modelsNames, rotation='vertical')

plt.figure(figsize=(4,2))
plt.bar(modelsNames, maleRatio)
plt.title(' Male Ratio: p(m|gender word) ')
plt.ylabel('bias')
_=plt.xticks(modelsNames, rotation='vertical')

In [ ]:
for i in range ( len(modelsNames)):
    plt.figure(figsize=(12,2))
    plt.title(modelsNames[i])
    _=plt.hist(scorePList[i], bins=100 , range=(0,3))

In [ ]:
for i in range ( len(modelsNames)):
    plt.figure(figsize=(12,2))
    plt.title(modelsNames[i])
    _=plt.hist(scorePCList[i], bins=100 , range=(0,3))